# Exercício 1 - Trabalho Prático 1

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Problema: 


1. Use a package **cryptography** para  criar um comunicação privada assíncrona entre um agente _Emitter_ e um agente _Receiver_ que cubra os seguintes aspectos:
    1. Comunicação cliente-servidor que use o package python `asyncio`.
    2. Usar como cifra AEAD   o “hash” SHAKE-256  em modo XOFHash
    3. As chaves de cifra  e  os “nounces” são gerados por um gerador KDF . As diferentes chaves para inicialização KDF  são inputs do emissor e do receptor.

# Implementação do Problema

**Import**
1. Instalar/importar as funcionalidades necessárias do `crypography`
2. Instalar/importar o `asyncio` para ser possível a criação do cliente-servidor assíncrono

In [2]:
%pip install cryptography asyncio
import asyncio

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

Note: you may need to restart the kernel to use updated packages.


Inicialmente é necessário criar uma variável global pra garantir que os nounces gerados são únicos e não se repetem

In [14]:
nounce_list=[]

### Funções Importantes

1. É definido a função "shake256XOF" com return do hash final
    1. Verificação se a strig passada está em bytes
    2. Executar o código que implementa o SHAKE256XOF

Esta função iniciliza o modelo sponge, sendo seguido do `absorve` e do `squeeze`, o que implica que é `XOF` 
    
[Fonte - Documentação de "cryptography"](https://cryptography.io/en/latest/hazmat/primitives/cryptographic-hashes/)

In [15]:
def shake256XOF(text , length=32):
    if isinstance(text, str):
        text = text.encode('utf-8')
    elif not isinstance(text, bytes):
        raise TypeError("Input must be string or bytes")
    
    
    digest = hashes.Hash(hashes.SHAKE256(length),  backend=default_backend()) #sponge
    digest.update(text) #Absorve
    return digest.finalize() #Squeeze

1. É definido a função "derive_nounce" com return do nounce final
    1. Verificação se a strig passada está em bytes
    2. Executar 

Função muito semelhante à anterior, mas desta vez usando SHA256

[Fonte - Documentação de "cryptography"](https://cryptography.io/en/latest/development/custom-vectors/hkdf/#hkdf-vector-creation)

In [16]:

def derive_nounce(text: bytes):
    if isinstance(text, str):
        text = text.encode('utf-8')
    elif not isinstance(text, bytes):
        raise TypeError("Input must be string or bytes")

    #Sponge
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=12,
        salt=None,
        info=b"Nonce Generation",
        backend=default_backend()
    )
    return hkdf.derive(text) #Absorve n Squeeze


1. É definido a função "cipher_ex1" para cifrar
    1. Verificação se a strig passada está em bytes
    2. Obter um `nounce` **único** evita ataques de repitação para os mesmos _inputs_ dando origem a um `keystream` **único**
    3. É gerado o `keystream` usando a função previamente criada (_shake256XOF_)
    4. É criado o `ciphertext` aplicando `XOR` entre o _plaintext_ e _keystream_. A utilização permite que seja facilmente revertido, **SENDO FUNDAMENTAL**, que o _keystream_ seja não reutilizado (unicicidade)
    5. `TAG` - É aplicado `shake256XOF` com o a concatenação entre _key_, _nounce_, _ciphertext_ para garantir que é uma cifra `AEAD` e assim evitar ataques de modificação obtendo assim conhecimento se existir uma `violação da integridade` da mensagem

[Fonte - Estruturas Criptograficas UM](https://www.dropbox.com/scl/fi/g5vq72hi2fs3ceny7u6bn/Estruturas-Criptograficas-2024-2025.paper?rlkey=0aklld7aud44twg9yepzjo9mf&dl=0)

In [17]:
def cipher_ex1(plaintext: str, key: bytes):
    global nounce_list
    
    if isinstance(plaintext, str):
        plaintext = plaintext.encode('utf-8')
    
    nounce = derive_nounce(key)
    while nounce in nounce_list:
        nounce = shake256XOF(nounce)
    nounce_list.append(nounce)
    
    try:

        keystream = shake256XOF(key + nounce, length=len(plaintext))
        
        ciphertext = bytes(p ^ k for p, k in zip(plaintext, keystream))
        
        tag_input = key + nounce + ciphertext
        tag = shake256XOF(tag_input)
        
        return nounce, ciphertext, tag
        
    except Exception as e:
        print(e)
        return None


1. É definido a função "de_cipher_ex1" para decifrar
    1. `TAG` - É aplicado `shake256XOF` com o a concatenação entre _key_, _nounce_, _ciphertext_ para garantir que é uma cifra `AEAD` e assim evitar ataques de modificação obtendo assim conhecimento se existir uma `violação da integridade` da mensagem.
    2. Verificar se efetivamente a mensagem sofreu `violação de integrdidade`
    3. Se não sofreu, é necssário voltar a gerar a `keystream` tendo em cosideração que o hashing é um processo deterministico.
    4. É descuberto o `plaintext` aplicando `XOR` entre o _ciphertext_ e _keystream_

[Fonte - Estruturas Criptograficas UM](https://www.dropbox.com/scl/fi/g5vq72hi2fs3ceny7u6bn/Estruturas-Criptograficas-2024-2025.paper?rlkey=0aklld7aud44twg9yepzjo9mf&dl=0)

In [18]:
def de_cipher_ex1(ciphertext: bytes, key: bytes, nounce: bytes, tag: bytes):
    try:
        
        tag_input = key + nounce + ciphertext
        expected_tag = shake256XOF(tag_input)
        
        if tag != expected_tag:
            raise ValueError("Authentication failed - Tag mismatch")

        keystream = shake256XOF(key + nounce, length=len(ciphertext))
        
        plaintext = bytes(c ^ k for c, k in zip(ciphertext, keystream))
        
        return plaintext
        
    except ValueError as e:
        print(e)
        return None
    except Exception as e:
        print(e)
        return None


### Emitter

É definido a função `Emitter` que envia o _input_ do utilizador encriptado seguindo as diretrizes pedidas no enunciado

A mensagem `"0"` permite desligar este programa

[Fonte - Documentação de "asyncio"](https://docs.python.org/3/library/asyncio.html)

In [19]:
async def emitter(queue,seed):
    key = shake256XOF(seed)
    print(f"Emitter Key: {key}")
    message_id = 0
    loop = asyncio.get_event_loop()
    while True:
        message=await loop.run_in_executor(None, input, "MESSAGE: ")

        if message=="0":
            for task in asyncio.all_tasks():
                task.cancel()  # Isso vai cancelar todas as tasks, incluindo a main
            break

        print("*"*50)
        print(f"{message_id} - ")
        print(f"Message From EMITTER: {message}")

        print("-"*20)
        out=cipher_ex1(message,key)

        message_id += 1
        await queue.put(out)
        

### Receiver

É definido a função `Receiver` que recebe o _input_ do utilizador encriptado e irá desencripta-lo mostrando a resposta final no terminal

A mensagem `"0"` permite desligar este programa

[Fonte - Documentação de "asyncio"](https://docs.python.org/3/library/asyncio.html)

In [20]:
async def receiver(queue,seed):
    key = shake256XOF(seed)
    print(f"Receiver Key: {key}")
    while True:
        nounce, ciphertext, tag = await queue.get()
        message=de_cipher_ex1(ciphertext,key,nounce, tag)
        if message=="0":
            for task in asyncio.all_tasks():
                task.cancel()  # Isso vai cancelar todas as tasks, incluindo a main
            break

        print(f"Received cipher: {ciphertext}")
        print(f"Received tag: {tag}")
        print(f"Received nounce: {nounce}")
        print(f"The real message: {message}")



# Run

As funções previamente criadas irão correr de forma assíncrona tendo como ponto de comunicação a class `Queue` da libraria _asyncio_

In [21]:
async def main():
    try:
        queue = asyncio.Queue()
        seed=input("Digite uma frase (seed):")

        # Emitter & Receiver Tasks
        emitter_task = asyncio.create_task(emitter(queue,seed))
        receiver_task = asyncio.create_task(receiver(queue, seed))

        await asyncio.gather(emitter_task, receiver_task) # Espera as tasks terminarem

        main_task = asyncio.create_task(main()) # Para matar a main task

        await main_task
    except asyncio.CancelledError:
        print("\n\nAcabou!")


In [22]:
await main()

Emitter Key: b'\xd0\xef\t\x14Z\x83(\xcd\nz\xc3\x90\xc6o\xb6^\x88\x9c\x0e\n\xe3\xc9\x13qj\x1b\x11\xbf\xe8\xdb_\xe8'
Receiver Key: b'\xd0\xef\t\x14Z\x83(\xcd\nz\xc3\x90\xc6o\xb6^\x88\x9c\x0e\n\xe3\xc9\x13qj\x1b\x11\xbf\xe8\xdb_\xe8'
**************************************************
0 - 
Message From EMITTER: Ola, o meu nome e pedro e gosto muito de voar
--------------------
Received cipher: b'e\xfd_\xb9\xdf\xd2\xca\xa7\x9d\x85\x81\x91\t\xa3\xeb\x88\xe5\xa4O;<\xba9\xc1\xf5\x04\xbb\xfc$Q\x18+\x16\xe8\x84y\x00\x96\x02|\xc35\x8a-@'
Received tag: b';\x02\x9fE%\xa3\xd1\x81\x80\xb6siq\x11\x1dB\xccL\x0b\xe9\x02\xc5u\xa6l\x9c8\xd3\xe09\xb3b'
Received nounce: b'\xf4\xf0\x97\xe4\x19#2\xf0K\xa7\xb1\xa8'
The real message: b'Ola, o meu nome e pedro e gosto muito de voar'
**************************************************
1 - 
Message From EMITTER: a serio???? Podemos simplesmente voar entao
--------------------
Received cipher: b'5\xc4\xc4N\x01\xec3\xf4\x8d\x82\x8a\x1ev\x8cp*s&tk\xf30\xf6\x89Y\xefOyX